In [ ]:
import numpy as np
import scipy.stats as stats
import os
import time
import plotly.graph_objects as go
import plotly.io as pio
from numba import njit, prange
from pathlib import Path


In [ ]:
# Parameters
MAX_T = 10000000
MAX_DF = 10000
NUM_QUANTILES = 99999
LIMIT = 100000000

# Precomputed values
sqrts = np.sqrt(np.arange(MAX_T))

# Load or compute sqare roots and quantiles
sqrts_path = f'tables/first_{MAX_T}_sqrts.npy'
n_quantiles_path = f'tables/n_quantiles_q{NUM_QUANTILES}.npy'
t_quantiles_path = f'tables/t_quantiles_df{MAX_DF}_q{NUM_QUANTILES}.npy'

# Ensure the 'data' directory exists
os.makedirs("data", exist_ok=True)

# Computes square roots
if not Path(sqrts_path).exists():
    sqrts = np.sqrt(np.arange(MAX_T))
    np.save(sqrts_path, sqrts)

# Computes normal and t distribution quantiles
if not Path(n_quantiles_path).exists() or not Path(t_quantiles_path).exists():
    
    # Possible quantiles for Thompson sampling
    quantiles = np.linspace(0.00001, 0.99999, NUM_QUANTILES).round(5)

    # Computes quantiles for normal and t distributions up to MAX_DF
    # after which normal approximation is used
    n_quantiles = stats.norm().ppf(quantiles)
    t_quantiles = np.array(
        [stats.t(df).ppf(quantiles) for df in range(1, MAX_DF + 1)], 
                  dtype=np.float32)

    # Saves the quantile tables
    np.save(n_quantiles_path, n_quantiles)
    np.save(t_quantiles_path, t_quantiles)

# Loads quantile tables and square roots
n_quantiles = np.load(n_quantiles_path)
t_quantiles = np.load(t_quantiles_path)
sqrts = np.load(sqrts_path)

In [ ]:
# A numba implementation for np.random.choice using the CDF of probabilites
@njit
def weighted_choice(probabilities, size):
    """A numba implementation for np.random.choice using the CDF of probabilites

    Args:
        probabilities (np.array): An array of probabilities corresponding to the
         probability of choosing that integer index. Should sum to 1.
        size (int): The number of samples to choose from the probability vector

    Returns:
        np.array: A vector of length (size) of integers chosen with the given 
        probabilites
    """
    cumulative_sum = np.cumsum(probabilities)
    choices = np.searchsorted(cumulative_sum, np.random.rand(size))
    return choices

In [ ]:
# Pre-determines rewards for each arm 
@njit
def compute_rewards(k, R, T):
    """Pre-computes rewards for each arm of the bandit at round t

    Args:
        k (int): The number of bandit arms.
        R (int): The range of reward.
        T (int): The number of rounds in the game.

    Returns:
        numpy vector: deltas : A length k vector containg the difference in mean 
        of each arm from the arm wih the highest mean.
        numpy matrix: arm_rewards: A k x T matrix that holds the rewards for 
        arm k at round T
    """

    # Randomizes reward probability for each arm
    p_vectors = np.random.dirichlet(np.ones(R), size=k)

    # Computes mean reward for each arm
    arm_true_means = np.dot(p_vectors, np.arange(R, dtype=np.float64))

    # Compute the difference in means from the best arm
    mu_star = arm_true_means.max()
    deltas = mu_star - arm_true_means
    
    # Pre determines rewards  for each arm 
    arm_rewards = np.empty((k, T), dtype=np.int32)
    for a in range(k):
        arm_rewards[a, :] = weighted_choice(p_vectors[a], T)

    return deltas, arm_rewards, arm_true_means


In [ ]:
@njit
def run_single_simulation(k, R, T, explore_lens,
                                   n_quantiles = n_quantiles, 
                                   t_quantiles = t_quantiles, 
                                   sqrts = sqrts):
    """Runs a single simulation of a multi-arm bandit with NUM_ALGO different
    algorithms.

    Args:
        k (int): The number of arms.
        R (int): The range of reward.
        T (int): The number of rounds.
        n_quantiles (np.array): A vector of quantiles from the normal 
        distribution.
        t_quantiles (np.array): A matrix of quantiles from the t distribution. 
        Size MAX_DF x NUM_QUANTILES.
        sqrts (np.array): A vector of square roots of the indices.

    Returns:
        np.matrix: A NUM_ALGO x T matrix containing the cumulative regrets for 
        Thompson sampling using a normal and t distribution.
    """
    # Length of explorations
    NUM_ALGO = explore_lens.size

    # Get the arm means and rewards
    deltas, arm_rewards, arm_true_means = compute_rewards(k, R, T)
    
    # Pre-sets quantiles for each round and arm
    arm_quantiles = np.random.randint(0, NUM_QUANTILES, (T, k))

    # Gets the quantiles for the normal distribution of each round and arm
    ts_n_quantiles = np.empty((T, k), dtype=n_quantiles.dtype)
    for t in range(T):
        for a in range(k):
            ts_n_quantiles[t, a] = n_quantiles[arm_quantiles[t, a]]

    # Initializes vectors to hold data during simulation
    ts_quantiles = np.empty(k, dtype=np.float32) # Thompson Sampled quantiles
    arm_means = np.zeros((NUM_ALGO, k)) # Running arm means
    arm_stds = np.zeros((NUM_ALGO, k)) +.000001 # Running arm standard deviations
    n_t = np.zeros((NUM_ALGO, k), dtype=np.int32) # Number of arm pulls
    R_t = np.zeros((NUM_ALGO, T), dtype=np.float32) # Cumulative regrets
    regrets = np.empty(NUM_ALGO, dtype=np.float32) # Regret of current round

    explore_idxs = [i for i in range(NUM_ALGO)]
    ts_idxs = []

    # Starts the simulation
    for t in range(T):

        # Initializes array to hold actions for each algorithm in the round
        actions = np.zeros(NUM_ALGO, dtype=np.int32)
        
        # For algorithms still in exploration phase, explore
        for idx in explore_idxs.copy():

            if t < explore_lens[idx] * k:
                actions[idx] = t % k

            elif t == explore_lens[idx] * k:
                ts_idxs.append(explore_idxs[0])
                explore_idxs = explore_idxs[1:]
        
        # Then Thompson Sample
        for idx in ts_idxs:
        
            if idx == 1 or idx == 3:
                # For each arm
                for a in range(k):

                    # If it's been pulled less than MAX_DF, use a T distribution
                    if n_t[1, a] <= MAX_DF + 1:
                        ts_quantiles[a] = (
                            t_quantiles[n_t[1, a] - 2, arm_quantiles[t, a]])

                    # Otherwise use a normal approximation
                    else:
                        ts_quantiles[a] = ts_n_quantiles[t,a]

            else:
                ts_quantiles = ts_n_quantiles[t,:]


            # Scales the normal and t distribution algorithms with mean and std
            adjusted_stds = arm_stds[idx] / sqrts[n_t[idx]]

            scaled_quantiles = (arm_means[idx] 
                                  + ts_quantiles * adjusted_stds)

            # Choose the action with the highest sampled mean
            actions[idx] = np.argmax(scaled_quantiles)

        # For each algorithm, update the mean and variance of the pulled arm
        for i in range(NUM_ALGO):

            # Store intermediate data
            action = actions[i]
            reward = arm_rewards[action, n_t[i, action]]
            regrets[i] = deltas[action]
            n_t[i, action] += 1
            current_mean = arm_means[i, action]

            # Intermediate calculations
            current_var = arm_stds[i, action] ** 2
            delta1 = reward - current_mean
            new_mean = current_mean + (delta1 / n_t[i, action])
            delta2 = reward - new_mean
            new_var = current_var + (delta1 * delta2 / n_t[i, action])

            # Update new mean and variance
            arm_means[i, action] = new_mean
            arm_stds[i, action] = np.sqrt(new_var)

        # Update the sequence of cumulative regrets
        if t > 0:
            R_t[:, t] = R_t[:, t - 1] + regrets
        else:
            R_t[:, t] = regrets
    
    
    return R_t


In [ ]:
@njit(parallel=True)
def run_parallel_simulation(N, k, R, T, explore_lens,
                            n_quantiles = n_quantiles, 
                            t_quantiles = t_quantiles, 
                            sqrts = sqrts):
    """Run N simulations of the multiarm bandits in parallel with Numba.

    Args:
        N (int): The number of simulations.
        k (int): The number of arms.
        R (int): The range of reward.
        T (int): The number of rounds.
        n_quantiles (np.array): A vector of quantiles from the normal 
        distribution.
        t_quantiles (np.array): A matrix of quantiles from the t distribution. 
        Size MAX_DF x NUM_QUANTILES.
        sqrts (np.array): A vector of square roots of the indices.

    Returns:
        np matrix: A NUM_ALGO x T x N matrix containing the cumulative regrets
        of each algorithm over T rounds over N simulations
    """
    NUM_ALGO = explore_lens.size
    results = np.empty((NUM_ALGO, T, N), dtype=np.float32)
    for n in prange(N):
        results[:, :, n] = run_single_simulation(
                                k, R, T, explore_lens,
                                n_quantiles = n_quantiles, 
                                t_quantiles = t_quantiles, 
                                sqrts = sqrts)
    return results

In [ ]:
def run_simulations(N, k, R, T, explore_lens,
                    n_quantiles = n_quantiles, 
                    t_quantiles = t_quantiles, 
                    sqrts = sqrts):
    """Runs N simulations of the mulitarm bandits. Splits simulations into
    batches for better performance with high values of N and T.

    Args:
        N (int): The number of simulations.
        k (int): The number of arms.
        R (int): The range of reward.
        T (int): The number of rounds.
        n_quantiles (np.array): A vector of quantiles from the normal 
        distribution.
        t_quantiles (np.array): A matrix of quantiles from the t distribution. 
        Size MAX_DF x NUM_QUANTILES.
        sqrts (np.array): A vector of square roots of the indices.

    Returns:
        np matrix: A NUM_ALGO x T x N matrix containing the cumulative regrets
        of each algorithm over T rounds over N simulations
    """
    NUM_ALGO = explore_lens.size

    # Computes number of batches to run simulations in
    batches = N * T  // LIMIT

    start = time.time()

    if batches > 1:

        # Initialize arrays to hold running mean and variances of the data
        R_t_mean = np.zeros((NUM_ALGO, T))
        R_t_m2 = np.zeros((NUM_ALGO, T))  

        print(f"\nStarting {batches} batches.\n")

        # For each batch
        for i in range(batches):
            
            batch_start = time.time()
            
            # Run batch simulation and calculate mean and std for current batch
            R_t_all = run_parallel_simulation(
                        N // batches, k, R, T, explore_lens,
                                n_quantiles = n_quantiles, 
                                t_quantiles = t_quantiles, 
                                sqrts = sqrts)
            batch_mean = R_t_all.mean(axis=2)
            batch_std = R_t_all.std(axis=2)
            batch_variance = batch_std**2

            # Update the combined mean and variance iteratively
            delta = batch_mean - R_t_mean
            R_t_mean += delta / (i + 1)  
            R_t_m2 += batch_variance + delta**2 * i / (i + 1)  

            # Print logging info
            batch_end = time.time()
            elapsed = round(batch_end - batch_start)
            remaining = round(((elapsed * batches)-(elapsed * (i + 1))) / 60, 2)
            print(f"Batch {i + 1} complete, {elapsed} seconds.")
            print(f"Remaining: {remaining} minutes.\n")

        # Calculate the final standard deviation from the accumulated R_t_m2
        R_t_std = np.sqrt(R_t_m2 / batches)

    # If one batch is needed, run the simulation as normal
    else:
        R_t_all = run_parallel_simulation(
                                N, k, R, T, explore_lens,
                                n_quantiles = n_quantiles, 
                                t_quantiles = t_quantiles, 
                                sqrts = sqrts)
        R_t_mean = R_t_all.mean(axis=2)
        R_t_std = R_t_all.std(axis=2)

    # Save data
    np.save(f"data/k{k}_R{R}_N{N}_T{T}_Means", R_t_mean)
    np.save(f"data/k{k}_R{R}_N{N}_T{T}_Stds", R_t_std)

    # Print logging info
    end = time.time()
    elapsed = round((end - start) / 60, 2)
    print(f"Completion Time: {elapsed} minutes\n")

    return R_t_mean, R_t_std

In [ ]:
N = 10000
T = 1000000
explore_lens = np.array([2,2,15,30])
ks = [3,4,5,6,7,8]
Rs = [10,100,1000]

for k in ks:
    for R in Rs:
        run_simulations(N, k, R, T, explore_lens)

In [ ]:
N = 100
T = 10000
k = 3
R = 10
explore_lens = np.array([2,2,15,30])

R_t_mean, R_t_std = run_simulations(N, k, R, T, explore_lens)

In [ ]:
# Plot Results
fig_mean = go.Figure()
x_vals = np.arange(1,T+1)

fig_mean.add_trace(go.Scatter(x=x_vals, y=R_t_mean[0], mode = 'lines', name = 'Normal Distribution'))
fig_mean.add_trace(go.Scatter(x=x_vals, y=R_t_mean[1], mode = 'lines', name = 'T Distribution'))
fig_mean.add_trace(go.Scatter(x=x_vals, y=R_t_mean[2], mode = 'lines', name = 'Explore 15'))
fig_mean.add_trace(go.Scatter(x=x_vals, y=R_t_mean[3], mode = 'lines', name = 'Explore 30'))

fig_mean.update_layout(
    title=f"Mean Regret Across {N} Trials",
    title_x = 0.5,
    xaxis_title="Round",
    yaxis_title="Mean Regret",
    width=800,
    height=400,
    template="plotly",
    legend=dict(
        x=0.72,
        y=0.07,
        bgcolor="rgba(255, 255, 255, 0.25)" 
    )
)
fig_mean.show()

In [ ]:
fig_var = go.Figure()
x_vals = np.arange(1,T+1)

fig_var.add_trace(go.Scatter(x=x_vals, y=R_t_std[0], mode = 'lines', name = 'Normal Distribution'))
fig_var.add_trace(go.Scatter(x=x_vals, y=R_t_std[1], mode = 'lines', name = 'T Distribution'))
fig_var.add_trace(go.Scatter(x=x_vals, y=R_t_std[2], mode = 'lines', name = 'Explore 15'))
fig_var.add_trace(go.Scatter(x=x_vals, y=R_t_std[3], mode = 'lines', name = 'Explore 30'))

fig_var.update_layout(
    title=f"Variance of Regret across {N} Trials",
    title_x = 0.5,
    xaxis_title="Round",
    yaxis_title="Regret Variance",
    width=800,
    height=400,
    template="plotly",
    legend=dict(
        x=0.72,
        y=0.07,
        bgcolor="rgba(255, 255, 255, 0.25)" 
    )
)
fig_var.show()

In [ ]:
k = 3
R = 10
T = 10000

R_t = run_single_simulation(k, R, T, n_quantiles, t_quantiles, sqrts)

# Plot Results
fig_mean = go.Figure()
x_vals = np.arange(1,T+1)

fig_mean.add_trace(go.Scatter(x=x_vals, y=R_t[0], mode = 'lines', name = 'Normal Distribution',showlegend = True))
fig_mean.add_trace(go.Scatter(x=x_vals, y=R_t[1], mode = 'lines', name = 'T Distribution', showlegend = True))
fig_mean.add_trace(go.Scatter(x=x_vals, y=R_t[2], mode = 'lines', name = 'Explore 15',showlegend = True))
fig_mean.add_trace(go.Scatter(x=x_vals, y=R_t[3], mode = 'lines', name = 'Explore 30', showlegend = True))


fig_mean.update_layout(
    title="Regret of Thompson Sampling with Normal Posterior",
    title_x = 0.5,
    xaxis_title="Round",
    yaxis_title="Cumulative Regret",
    width=800,
    height=400,
    template="plotly",
    legend=dict(
        x=0.72,
        y=0.07,
        bgcolor="rgba(255, 255, 255, 0.25)" 
    )
)

fig_mean.show()

In [ ]:
# Loads and displays previously stored data
k = 3
R = 10
N = 10000
T = 1000000

R_t_mean = np.load(f"data/k{k}_R{R}_N{N}_T{T}_Means.npy")
R_t_std = np.load(f"data/k{k}_R{R}_N{N}_T{T}_Stds.npy")

# Plot Results
fig_mean = go.Figure()
x_vals = np.arange(1,T+1)

fig_mean.add_trace(go.Scatter(x=x_vals, y=R_t_mean[0], mode = 'lines', name = 'Normal Distribution'))
fig_mean.add_trace(go.Scatter(x=x_vals, y=R_t_mean[1], mode = 'lines', name = 'T Distribution'))
fig_mean.add_trace(go.Scatter(x=x_vals, y=R_t_mean[2], mode = 'lines', name = 'Explore 15, Normal'))
fig_mean.add_trace(go.Scatter(x=x_vals, y=R_t_mean[3], mode = 'lines', name = 'Explore 30, Normal'))

fig_mean.update_layout(
    title=f"Mean Regret Across {N} Trials",
    title_x = 0.5,
    xaxis_title="Round",
    yaxis_title="Mean Regret",
    width=800,
    height=400,
    template="plotly",
    legend=dict(
        x=0.01,
        y=0.97,
        bgcolor="rgba(255, 255, 255, 0.25)" 
    )
)

fig_mean.add_annotation(
    text=f"k = {k} ; R = {R}",
    xref="paper", yref="paper",
    x=0.5, y=1.15,  # Position the note
    showarrow=False,
    font=dict(size=14)
)
fig_mean.show()

In [ ]:
fig_var = go.Figure()
x_vals = np.arange(1,T+1)

fig_var.add_trace(go.Scatter(x=x_vals, y=R_t_std[0], mode = 'lines', name = 'Normal Distribution'))
fig_var.add_trace(go.Scatter(x=x_vals, y=R_t_std[1], mode = 'lines', name = 'T Distribution'))
fig_var.add_trace(go.Scatter(x=x_vals, y=R_t_std[2], mode = 'lines', name = 'Explore 15, Normal'))
fig_var.add_trace(go.Scatter(x=x_vals, y=R_t_std[3], mode = 'lines', name = 'Explore 30, Normal'))

fig_var.update_layout(
    title=f"Variance of Regret Across {N} Trials",
    title_x = 0.5,
    xaxis_title="Round",
    yaxis_title="Regret Variance",
    width=800,
    height=400,
    template="plotly",
    legend=dict(
        x=0.01,
        y=0.97,
        bgcolor="rgba(255, 255, 255, 0.25)" 
    )
)

fig_var.add_annotation(
    text=f"k = {k} ; R = {R}",
    xref="paper", yref="paper",
    x=0.5, y=1.15,  # Position the note
    showarrow=False,
    font=dict(size=14)
)
fig_var.show()